In [332]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm

## lấy mfcc của file wav

In [333]:
# read file
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

## lấy mfcc của tất cả các file trong dir

In [334]:
# lấy mfcc của tất cả các file wav trong wav
def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    return mfcc

## Hàm Clustering

In [335]:
def clustering(X, n_clusters=14):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans

## Data

In [336]:
class_names = ['bệnh nhân', 'chúng ta', 'có thể', 'người','Việt Nam']
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join(cname))

Load bệnh nhân dataset
Load chúng ta dataset
Load có thể dataset
Load người dataset
Load Việt Nam dataset


In [337]:
print(len(dataset['Việt Nam']))

100


## split train test

In [338]:
trainset = {}
testset = {}
n_test = {'bệnh nhân': 0, 'chúng ta': 0, 'có thể': 0, 'người': 0,'Việt Nam': 0}
for cname in class_names:
    n = len(dataset[cname])
    n_train = math.floor(n*0.7)
    trainset[cname] = dataset[cname][:n_train]
    testset[cname] = dataset[cname][n_train:]
    n_test[cname] += len(testset[cname])
    
print(len(trainset['Việt Nam']))

70


## Fit kmeans trên tập train

In [339]:
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in trainset.items()], axis=0)
print("vectors", all_vectors.shape)
# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors)
print("centers", kmeans.cluster_centers_.shape)
print(kmeans)

vectors (12818, 36)
centers (14, 36)
centers (14, 36)
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=14, n_init=50, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)


In [340]:
for cname in class_names:
    trainset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in trainset[cname]])
    testset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in testset[cname]])

    

# Train

In [341]:
models = {}

## Model cho 'bệnh nhân' 6x3

In [342]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=6*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xbn = np.concatenate(trainset['bệnh nhân'])
lengths = list([len(x) for x in trainset['bệnh nhân']])
print("training class", 'bệnh nhân')
print(Xbn.shape, lengths, len(lengths))
hmm.fit(Xbn, lengths=lengths)
models['bệnh nhân'] = hmm
print("Training done")

training class bệnh nhân
(2799, 1) [39, 29, 38, 32, 55, 48, 41, 41, 41, 47, 51, 43, 44, 63, 37, 35, 44, 40, 32, 41, 32, 41, 41, 31, 40, 44, 45, 30, 38, 35, 45, 48, 32, 35, 46, 33, 37, 51, 38, 45, 35, 35, 33, 29, 37, 47, 42, 43, 47, 44, 35, 43, 54, 41, 33, 36, 30, 33, 41, 43, 46, 41, 37, 40, 45, 26, 28, 52, 41, 34] 70


         1       -7298.5265             +nan
         2       -5464.2509       +1834.2756
         3       -4810.7892        +653.4617
         4       -4444.7017        +366.0875
         5       -4284.8125        +159.8892
         6       -4201.2031         +83.6094
         7       -4156.7224         +44.4807
         8       -4123.4403         +33.2821
         9       -4080.7503         +42.6901
        10       -4016.9129         +63.8374
        11       -3960.8653         +56.0476
        12       -3932.0619         +28.8034
        13       -3918.7213         +13.3406
        14       -3911.5884          +7.1329
        15       -3900.6868         +10.9016
        16       -3894.8885          +5.7984
        17       -3890.6452          +4.2433
        18       -3887.3914          +3.2538
        19       -3886.3481          +1.0432
        20       -3885.5917          +0.7565
        21       -3884.9277          +0.6640
        22       -3884.2496          +0.6781
        23

Training done


       122       -3842.3269          +0.0106
       123       -3842.3169          +0.0100
       124       -3842.3073          +0.0095


## Model cho 'chúng ta' 5x3

In [343]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=5*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xct = np.concatenate(trainset['chúng ta'])
lengths = list([len(x) for x in trainset['chúng ta']])
print("training class", 'chúng ta')
print(Xct.shape, lengths, len(lengths))
hmm.fit(Xct, lengths=lengths)
models['chúng ta'] = hmm
print("Training done")

training class chúng ta
(2802, 1) [34, 31, 34, 75, 33, 35, 59, 39, 36, 38, 37, 37, 41, 42, 41, 35, 34, 37, 34, 32, 31, 40, 30, 37, 38, 39, 38, 55, 35, 54, 40, 40, 52, 36, 43, 37, 44, 41, 37, 39, 50, 32, 46, 47, 38, 46, 42, 39, 36, 41, 47, 28, 36, 47, 41, 43, 32, 41, 38, 42, 36, 45, 30, 32, 36, 48, 43, 57, 37, 36] 70


         1       -6924.3953             +nan
         2       -4745.0425       +2179.3528
         3       -4025.0640        +719.9785
         4       -3742.6624        +282.4016
         5       -3627.1658        +115.4966
         6       -3570.5032         +56.6626
         7       -3541.0944         +29.4088
         8       -3524.1100         +16.9844
         9       -3512.6242         +11.4857
        10       -3504.0234          +8.6008
        11       -3497.6652          +6.3583
        12       -3492.8884          +4.7768
        13       -3486.0020          +6.8863
        14       -3477.6814          +8.3206
        15       -3471.2387          +6.4427
        16       -3466.8713          +4.3674
        17       -3464.5364          +2.3349
        18       -3462.7782          +1.7582
        19       -3460.7087          +2.0695
        20       -3458.2817          +2.4270
        21       -3456.0639          +2.2178
        22       -3454.5820          +1.4819
        23

Training done


        50       -3445.3895          +0.0103
        51       -3445.3806          +0.0089


## Model cho 'có thể' 4x3

In [344]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=4*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xcth = np.concatenate(trainset['có thể'])
lengths = list([len(x) for x in trainset['có thể']])
print("training class", 'có thể')
print(Xcth.shape, lengths, len(lengths))
hmm.fit(Xcth, lengths=lengths)
models['có thể'] = hmm
print("Training done")

         1       -6693.6318             +nan
         2       -5072.0862       +1621.5456


training class có thể
(2600, 1) [32, 40, 57, 23, 41, 43, 38, 102, 36, 37, 62, 25, 35, 37, 53, 36, 34, 44, 35, 51, 35, 39, 39, 39, 28, 31, 39, 37, 40, 44, 52, 46, 41, 52, 26, 47, 36, 52, 59, 43, 43, 30, 28, 36, 38, 35, 45, 31, 30, 56, 39, 53, 34, 34, 26, 56, 30, 33, 44, 37, 36, 35, 40, 25, 20] 65


         3       -4585.9863        +486.0998
         4       -4421.5962        +164.3902
         5       -4353.0428         +68.5533
         6       -4297.7195         +55.3233
         7       -4238.9855         +58.7341
         8       -4191.1757         +47.8097
         9       -4149.1614         +42.0144
        10       -4117.0518         +32.1095
        11       -4092.1871         +24.8647
        12       -4069.2819         +22.9053
        13       -4041.0039         +28.2780
        14       -4014.2235         +26.7804
        15       -3995.2368         +18.9867
        16       -3985.9655          +9.2714
        17       -3979.3970          +6.5685
        18       -3972.1856          +7.2114
        19       -3968.1633          +4.0223
        20       -3966.6235          +1.5398
        21       -3964.5656          +2.0579
        22       -3962.4976          +2.0679
        23       -3961.8415          +0.6561
        24       -3961.3647          +0.4768
        25

Training done


        57       -3954.6611          +0.0079


## Model cho 'người' 3x3

In [345]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=3*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xcth = np.concatenate(trainset['người'])
lengths = list([len(x) for x in trainset['người']])
print("training class", 'người')
print(Xcth.shape, lengths, len(lengths))
hmm.fit(Xcth, lengths=lengths)
models['người'] = hmm
print("Training done")

         1       -4847.2001             +nan
         2       -3591.9842       +1255.2159
         3       -3324.9737        +267.0105
         4       -3171.4964        +153.4773


training class người
(1923, 1) [30, 58, 16, 36, 26, 33, 27, 18, 18, 22, 29, 22, 22, 31, 23, 32, 25, 15, 20, 21, 23, 29, 28, 37, 18, 24, 27, 62, 21, 23, 27, 16, 26, 35, 13, 20, 20, 20, 28, 16, 19, 43, 23, 18, 42, 50, 28, 20, 38, 23, 38, 22, 22, 36, 37, 18, 22, 21, 24, 20, 25, 22, 23, 25, 23, 30, 22, 19, 42, 25, 19, 26, 21] 73


         5       -3063.5528        +107.9435
         6       -3002.9919         +60.5609
         7       -2966.5754         +36.4165
         8       -2922.9265         +43.6489
         9       -2906.7462         +16.1803
        10       -2902.5311          +4.2152
        11       -2899.8503          +2.6808
        12       -2898.2133          +1.6370
        13       -2897.0344          +1.1789
        14       -2896.0044          +1.0300
        15       -2895.0295          +0.9749
        16       -2894.1335          +0.8959
        17       -2893.3780          +0.7555
        18       -2892.7847          +0.5933
        19       -2892.3240          +0.4607
        20       -2891.9321          +0.3920
        21       -2891.5294          +0.4026
        22       -2891.0391          +0.4904
        23       -2890.4171          +0.6220
        24       -2889.5881          +0.8290
        25       -2888.1894          +1.3987
        26       -2885.2525          +2.9369
        27

Training done


        48       -2854.7178          +0.0059


## Model cho 'Việt Nam' 6x3

In [346]:
hmm = hmmlearn.hmm.MultinomialHMM(n_components=6*3, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
hmm.transmat_ =np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])

Xvn = np.concatenate(trainset['Việt Nam'])
lengths = list([len(x) for x in trainset['Việt Nam']])
print("training class", 'Việt Nam')
print(Xvn.shape, lengths, len(lengths))
hmm.fit(Xvn, lengths=lengths)
models['Việt Nam'] = hmm
print("Training done")

         1       -6919.4277             +nan


training class Việt Nam
(2694, 1) [26, 41, 42, 44, 56, 27, 38, 35, 44, 44, 32, 38, 36, 44, 57, 37, 41, 39, 34, 27, 40, 37, 38, 37, 34, 57, 44, 26, 39, 26, 32, 57, 35, 28, 35, 35, 60, 36, 47, 27, 41, 34, 59, 42, 41, 35, 37, 27, 33, 37, 46, 37, 41, 36, 35, 28, 31, 30, 40, 36, 37, 35, 44, 39, 31, 47, 37, 43, 50, 33] 70


         2       -4816.5216       +2102.9060
         3       -4262.2830        +554.2386
         4       -4005.0021        +257.2809
         5       -3875.1268        +129.8753
         6       -3800.0143         +75.1125
         7       -3747.3032         +52.7112
         8       -3714.5441         +32.7590
         9       -3696.4422         +18.1019
        10       -3685.2818         +11.1604
        11       -3677.0188          +8.2629
        12       -3667.7616          +9.2572
        13       -3651.2672         +16.4944
        14       -3618.8655         +32.4017
        15       -3578.0238         +40.8417
        16       -3542.3173         +35.7066
        17       -3516.1937         +26.1235
        18       -3493.3807         +22.8131
        19       -3475.6075         +17.7732
        20       -3461.0145         +14.5929
        21       -3456.3814          +4.6331
        22       -3455.4491          +0.9324
        23       -3455.1674          +0.2816
        24

Training done


        38       -3454.5015          +0.0092


# Test

In [347]:
print("Testing")
n_correct = {'bệnh nhân': 0, 'chúng ta': 0, 'có thể': 0, 'người': 0,'Việt Nam': 0}
for true_cname in class_names:
    for O in testset[true_cname]:
        score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
        if (true_cname == max(score, key=score.get)): n_correct[true_cname] += 1
        print(true_cname, score, 'predict:', max(score, key=score.get))

Testing
bệnh nhân {'bệnh nhân': -65.13425521189181, 'chúng ta': -400.1965661667665, 'có thể': -109.2039863897304, 'người': -763.5497105683475, 'Việt Nam': -826.5966443818475} predict: bệnh nhân
bệnh nhân {'bệnh nhân': -63.68115382019239, 'chúng ta': -233.0568302833083, 'có thể': -124.17113663673008, 'người': -116.07159248651006, 'Việt Nam': -246.3331424958047} predict: bệnh nhân
bệnh nhân {'bệnh nhân': -56.195767849806224, 'chúng ta': -99.83120681254502, 'có thể': -100.54151204495471, 'người': -81.90788443877902, 'Việt Nam': -170.96922371129844} predict: bệnh nhân
bệnh nhân {'bệnh nhân': -46.770470669632736, 'chúng ta': -108.1210036600973, 'có thể': -106.59830654983632, 'người': -123.77719092523321, 'Việt Nam': -108.6661626219529} predict: bệnh nhân
bệnh nhân {'bệnh nhân': -49.967061183307166, 'chúng ta': -134.50461658686726, 'có thể': -86.02675167928328, 'người': -313.19358456033046, 'Việt Nam': -322.4486864529609} predict: bệnh nhân
bệnh nhân {'bệnh nhân': -60.987885348460665, 'chúng

có thể {'bệnh nhân': -414.65432587724183, 'chúng ta': -237.0245066661402, 'có thể': -57.64993055598486, 'người': -1272.1864797114588, 'Việt Nam': -480.0639957451657} predict: có thể
có thể {'bệnh nhân': -86.00580701679635, 'chúng ta': -156.33880907744467, 'có thể': -62.354603844057465, 'người': -96.74704311795288, 'Việt Nam': -257.77741885671355} predict: có thể
có thể {'bệnh nhân': -44.386496585097085, 'chúng ta': -41.3345262951738, 'có thể': -27.160910762375188, 'người': -inf, 'Việt Nam': -38.94805419799321} predict: có thể
có thể {'bệnh nhân': -inf, 'chúng ta': -188.94283254274777, 'có thể': -38.91439071114712, 'người': -262.7563466152299, 'Việt Nam': -inf} predict: có thể
có thể {'bệnh nhân': -2937.966065722215, 'chúng ta': -239.18351542613863, 'có thể': -71.65761408890307, 'người': -5050.140259150939, 'Việt Nam': -705.6569191800247} predict: có thể
có thể {'bệnh nhân': -58.95400094853704, 'chúng ta': -66.21366444369735, 'có thể': -39.65888478533357, 'người': -182.03505665929157, '

Việt Nam {'bệnh nhân': -270.4601497044303, 'chúng ta': -37.19158632035436, 'có thể': -137.34680904039843, 'người': -61.074472165921854, 'Việt Nam': -29.023297514423845} predict: Việt Nam
Việt Nam {'bệnh nhân': -87.83212770924503, 'chúng ta': -60.442611031089434, 'có thể': -139.6003733874375, 'người': -100.23595855747317, 'Việt Nam': -39.71746023918096} predict: Việt Nam
Việt Nam {'bệnh nhân': -115.90942225777077, 'chúng ta': -67.20034352472058, 'có thể': -144.31593539754576, 'người': -94.95132021056747, 'Việt Nam': -53.656630494954534} predict: Việt Nam
Việt Nam {'bệnh nhân': -64.23597563375817, 'chúng ta': -48.827190943687285, 'có thể': -75.97874547734445, 'người': -45.62051256980294, 'Việt Nam': -33.068949070255634} predict: Việt Nam
Việt Nam {'bệnh nhân': -68.42826989249046, 'chúng ta': -96.96505430084174, 'có thể': -89.11367722053559, 'người': -202.12704698185308, 'Việt Nam': -46.50628901759651} predict: Việt Nam
Việt Nam {'bệnh nhân': -88.73864136731255, 'chúng ta': -141.040436323

In [348]:
for cname in class_names:
    print('Accuracy:', cname, n_correct[cname]/n_test[cname])

Accuracy: bệnh nhân 0.9
Accuracy: chúng ta 0.9333333333333333
Accuracy: có thể 0.9655172413793104
Accuracy: người 0.84375
Accuracy: Việt Nam 0.9666666666666667


In [349]:
print('All Accuracy:', sum(n_correct.values())/sum(n_test.values()))

All Accuracy: 0.9205298013245033


In [350]:
print(n_test)

{'bệnh nhân': 30, 'chúng ta': 30, 'có thể': 29, 'người': 32, 'Việt Nam': 30}


In [351]:
np.around(models['người'].transmat_, 2)

array([[0.83, 0.06, 0.11, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.82, 0.18, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.81, 0.06, 0.12, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.86, 0.14, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.91, 0.05, 0.04, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.98, 0.  , 0.02, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.96, 0.04, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.83, 0.17],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

In [352]:
np.around(models['có thể'].transmat_, 2)

array([[0.79, 0.21, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.87, 0.13, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.43, 0.02, 0.55, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.91, 0.09, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.67, 0.06, 0.27, 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.44, 0.56, 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.63, 0.01, 0.36, 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.8 , 0.11, 0.1 , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.51, 0.41, 0.09,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.82, 0.18,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.73,
        0.27],
       [0.  , 0.  , 0

In [353]:
np.around(models['bệnh nhân'].transmat_, 2)

array([[0.68, 0.11, 0.21, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.67, 0.33, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.59, 0.28, 0.13, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.7 , 0.3 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.63, 0.12, 0.25, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.84, 0.08, 0.07, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.81, 0.13, 0.05, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.79, 0.18, 0.04, 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],


In [354]:
np.around(models['Việt Nam'].transmat_, 2)

array([[0.74, 0.26, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.85, 0.06, 0.08, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.8 , 0.15, 0.04, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.85, 0.15, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.38, 0.1 , 0.52, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.87, 0.  , 0.13, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.46, 0.38, 0.16, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.77, 0.17, 0.05, 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],


In [355]:
np.around(models['chúng ta'].transmat_, 2)

array([[0.86, 0.12, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.82, 0.18, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.85, 0.15, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.77, 0.22, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.35, 0.24, 0.41, 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.88, 0.  , 0.12, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.12, 0.88, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.58, 0.39, 0.04, 0.  ,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.83, 0.  , 0.17,
        0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ,

## Test dữ liệu nói từ micro

In [356]:
testset1 = {}
n_test1 = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    testset1[cname] = get_class_data(os.path.join('test',cname+'_test'))
    n_test1[cname] = len(testset1[cname])

Load bệnh nhân dataset
Load chúng ta dataset
Load có thể dataset
Load người dataset
Load Việt Nam dataset


In [357]:
for cname in class_names:
    testset1[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in testset1[cname]])

In [358]:
print("Testing")
n_correct1 = {'bệnh nhân': 0, 'chúng ta': 0, 'có thể': 0, 'người': 0,'Việt Nam': 0}
for true_cname in class_names:
    for O in testset1[true_cname]:
        score = {cname: model.score(O, [len(O)]) for cname, model in models.items()}
        if (true_cname == max(score, key=score.get)): n_correct1[true_cname] += 1
        print(true_cname, score, 'predict:', max(score, key=score.get))

Testing
bệnh nhân {'bệnh nhân': -85.01895072882863, 'chúng ta': -116.89227689303209, 'có thể': -145.5429951589105, 'người': -79.12924074435324, 'Việt Nam': -105.07484469037391} predict: người
bệnh nhân {'bệnh nhân': -68.56648775916136, 'chúng ta': -135.8658249915291, 'có thể': -145.92926637729198, 'người': -109.12223911501658, 'Việt Nam': -130.38131951425976} predict: bệnh nhân
bệnh nhân {'bệnh nhân': -57.163946179657664, 'chúng ta': -184.1771335088101, 'có thể': -126.65149206275542, 'người': -80.46311942385488, 'Việt Nam': -76.78814830780043} predict: bệnh nhân
bệnh nhân {'bệnh nhân': -80.41844963773977, 'chúng ta': -185.13929588212147, 'có thể': -146.46293056268, 'người': -112.6877792505747, 'Việt Nam': -190.7548103572772} predict: bệnh nhân
bệnh nhân {'bệnh nhân': -64.49725977557267, 'chúng ta': -132.75405520685666, 'có thể': -103.81233439400432, 'người': -167.05841916754355, 'Việt Nam': -85.70916966073456} predict: bệnh nhân
chúng ta {'bệnh nhân': -1613.4628264667938, 'chúng ta': -

In [359]:
for cname in class_names:
    print('Accuracy:', cname, n_correct1[cname]/n_test1[cname])

Accuracy: bệnh nhân 0.8
Accuracy: chúng ta 0.8333333333333334
Accuracy: có thể 0.8
Accuracy: người 1.0
Accuracy: Việt Nam 1.0


In [360]:
print(n_test1)

{'bệnh nhân': 5, 'chúng ta': 6, 'có thể': 5, 'người': 5, 'Việt Nam': 6}


In [361]:
np.around(models['người'].transmat_, 2)

array([[0.83, 0.06, 0.11, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.82, 0.18, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.81, 0.06, 0.12, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.86, 0.14, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.91, 0.05, 0.04, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.98, 0.  , 0.02, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.96, 0.04, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.83, 0.17],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])